In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [2]:
import sys
sys.path.insert(0, "../../code-previous")

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import time
import os

import operator
import pickle
import string
import swifter
import nltk
stopwords = nltk.corpus.stopwords.words('english')


import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')



from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
disciplines = ["Architecture","Art","Humanities, Multidisciplinary","Folklore","Medieval & Renaissance Studies","Asian Studies","Classics","Dance","Film, Radio, Television","History","History & Philosophy of Science","Literary Theory & Criticism","Literary Reviews","Literature","Literature, African, Australian, Canadian","Literature, American","Literature, British Isles","Literature, German, Dutch, Scandinavian","Literature, Romance","Literature, Slavic","Poetry","Music","Philosophy","Religion","Theater","Agriculture, Dairy & Animal Science","Agricultural Engineering","Agricultural Economics & Policy","Agriculture, Multidisciplinary","Agronomy","Horticulture","Soil Science","Allergy","Anatomy & Morphology","Anesthesiology","Anthropology","Audiology & Speech-Language Pathology","Behavioral Sciences","Biochemical Research Methods","Biochemistry & Molecular Biology","Biodiversity Conservation","Biophysics","Biotechnology & Applied Microbiology","Cardiac & Cardiovascular Systems","Peripheral Vascular Disease","Cell & Tissue Engineering","Cell Biology","Dentistry, Oral Surgery & Medicine","Dermatology","Developmental Biology","Emergency Medicine","Andrology","Endocrinology & Metabolism","Entomology","Ecology","Environmental Sciences","Environmental Studies","Evolutionary Biology","Fisheries","Food Science & Technology","Forestry","Gastroenterology & Hepatology","Critical Care Medicine","Primary Health Care","Medicine, General & Internal","Genetics & Heredity","Geriatrics & Gerontology","Gerontology","Health Care Sciences & Services","Health Policy & Services","Hematology","Immunology","Infectious Diseases","Integrative & Complementary Medicine","Medicine, Legal","Biology","Limnology","Marine & Freshwater Biology","Mathematical & Computational Biology","Medical Ethics","Medical Informatics","Medical Laboratory Technology","Microbiology","Mycology","Clinical Neurology","Neurosciences","Neuroimaging","Nursing","Nutrition & Dietetics","Obstetrics & Gynecology","Oncology","Ophthalmology","Orthopedics","Otorhinolaryngology","Paleontology","Parasitology","Pathology","Pediatrics","Chemistry, Medicinal","Pharmacology & Pharmacy","Physiology","Plant Sciences","Psychiatry","Public, Environmental & Occupational Health","Radiology, Nuclear Medicine & Medical Imaging","Rehabilitation","Reproductive Biology","Medicine, Research & Experimental","Respiratory System","Rheumatology","Sport Sciences","Substance Abuse","Surgery","Toxicology","Transplantation","Tropical Medicine","Urology & Nephrology","Veterinary Sciences","Virology","Ornithology","Zoology","Astronomy & Astrophysics","Chemistry, Applied","Chemistry, Multidisciplinary","Chemistry, Analytical","Chemistry, Inorganic & Nuclear","Chemistry, Organic","Chemistry, Physical","Crystallography","Electrochemistry","Geochemistry & Geophysics","Geology","Geosciences, Multidisciplinary","Mathematics, Applied","Mathematics, Interdisciplinary Applications","Mathematics","Statistics & Probability","Meteorology & Atmospheric Sciences","Mineralogy","Mining & Mineral Processing","Oceanography","Optics","Geography, Physical","Physics, Applied","Physics, Fluids & Plasmas","Physics, Atomic, Molecular & Chemical","Physics, Multidisciplinary","Physics, Condensed Matter","Physics, Nuclear","Physics, Particles & Fields","Physics, Mathematical","Polymer Science","Thermodynamics","Water Resources","Nanoscience & Nanotechnology","Logic","Multidisciplinary Sciences","Acoustics","Automation & Control Systems","Computer Science, Artificial Intelligence","Computer Science, Cybernetics","Computer Science, Hardware & Architecture","Computer Science, Information Systems","Computer Science, Interdisciplinary Applications","Computer Science, Software Engineering","Computer Science, Theory & Methods","Construction & Building Technology","Energy & Fuels","Engineering, Aerospace","Engineering, Multidisciplinary","Engineering, Biomedical","Engineering, Environmental","Engineering, Chemical","Engineering, Industrial","Engineering, Manufacturing","Engineering, Marine","Engineering, Civil","Engineering, Ocean","Engineering, Petroleum","Engineering, Electrical & Electronic","Engineering, Mechanical","Engineering, Geological","Ergonomics","Imaging Science & Photographic Technology","Information Science & Library Science","Instruments & Instrumentation","Materials Science, Paper & Wood","Materials Science, Ceramics","Materials Science, Multidisciplinary","Materials Science, Biomaterials","Materials Science, Characterization & Testing","Materials Science, Coatings & Films","Materials Science, Composites","Materials Science, Textiles","Mechanics","Metallurgy & Metallurgical Engineering","Microscopy","Nuclear Science & Technology","Operations Research & Management Science","Remote Sensing","Robotics","Spectroscopy","Telecommunications","Transportation","Transportation Science & Technology","Archaeology","Area Studies","Social Sciences, Biomedical","Business","Business, Finance","Economics","Industrial Relations & Labor","Management","Communication","Criminology & Penology","Cultural Studies","Demography","Education & Educational Research","Education, Scientific Disciplines","Education, Special","Ethnic Studies","Family Studies","Geography","Law","Political Science","International Relations","Linguistics","Language & Linguistics","Social Sciences, Mathematical Methods","Psychology, Biological","Psychology, Clinical","Psychology, Educational","Psychology, Developmental","Psychology, Applied","Psychology","Psychology, Multidisciplinary","Psychology, Psychoanalysis","Psychology, Mathematical","Psychology, Experimental","Psychology, Social","Planning & Development","Public Administration","Social Issues","Ethics","History of Social Sciences","Hospitality, Leisure, Sport & Tourism","Social Sciences, Interdisciplinary","Social Work","Sociology","Urban Studies","Women's Studies"]

In [5]:
data = None

In [5]:
folder = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/word_selection_entropy"

with open(folder + "/data_grouped_by_label.pickle", 'rb') as f:
    data_grouped_by_label = pickle.load(f)

In [41]:
data_grouped_by_label["title_words_"] = data_grouped_by_label["title_without_stopwords"].progress_apply(lambda x: " ".join(x))

Progress: 100%|██████████| 249/249 [00:22<00:00, 11.05it/s]


In [42]:
def num_there(w):
    return any(i.isdigit() for i in w)


def clean_digits(s):
    t = s.split()
    cleaned = " ".join([x for x in t if not num_there(x)])
    return cleaned

data_grouped_by_label["title_words"] = data_grouped_by_label["title_words_"].progress_apply(lambda x: clean_digits(x))

Progress: 100%|██████████| 249/249 [16:45<00:00,  3.71s/it] 


In [43]:
data_grouped_by_label.head()

,title_without_stopwords,title_words,title_words_
Label,,,
Acoustics,"[multiplexer using flat exponential filters, m...",multiplexer using flat exponential filters mul...,multiplexer using flat exponential filters mul...
Agricultural Economics & Policy,"[poultry eggs, accounting commodity credit cor...",poultry eggs accounting commodity credit corpo...,poultry eggs accounting commodity credit corpo...
Agricultural Engineering,"[soft energy from palm wastes, potential land ...",soft energy from palm wastes potential land di...,soft energy from palm wastes potential land di...
"Agriculture, Dairy & Animal Science",[collection 2 cell embryos their culture media...,collection cell embryos their culture media co...,collection 2 cell embryos their culture media ...
"Agriculture, Multidisciplinary",[response strawberry fruit yield plant populat...,response strawberry fruit yield plant populati...,response strawberry fruit yield plant populati...


In [44]:
filename = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/corrected-entropy-analysis/all-data-grouped-by-label.pql"
pickle_out = open(filename,"wb")
pickle.dump(data_grouped_by_label, pickle_out)
pickle_out.close()

In [45]:
vectorizer = CountVectorizer()
vectorized_words = vectorizer.fit_transform(data_grouped_by_label.title_words)

In [46]:
pickle_out = open("/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/corrected-entropy-analysis/vectorizer.pickle","wb")
pickle.dump(vectorizer, pickle_out)
pickle_out.close()

pickle_out = open("/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/corrected-entropy-analysis/vectorized_words.pickle","wb")
pickle.dump(vectorized_words, pickle_out)
pickle_out.close()

In [47]:
count_vocab = {value:key for key,value in vectorizer.vocabulary_.items()}
count_rev_dict = {value:key for key,value in vectorizer.vocabulary_.items()}
words = [count_rev_dict[i] for i in range(len(vectorizer.vocabulary_.keys()))]

In [51]:
words[1000:1010]

['abaiang',
 'abailard',
 'abailles',
 'abaisse',
 'abaissee',
 'abaissement',
 'abaissements',
 'abaixo',
 'abaj',
 'abaja']

__Iterate through the columns of the vectorized words__

In [52]:
# https://stackoverflow.com/questions/19201972/can-numpy-bincount-work-with-2d-arrays

idx = 0
a = vectorized_words.transpose()
b = a.toarray()

In [53]:
b.shape

(2034183, 249)

In [55]:
np.amax(b)

493947

In [56]:
buffer = 5000
limit = np.amax(b) + 1

In [57]:
_unique, _b, _c, _unique_counts = np.unique(b[:1], return_index=True, return_inverse=True, return_counts=True, axis=0)
_unique_counts[0]

1

__Calculating entropies for every word from the vectorizer__

In [60]:
uniques = []
counts = []
entropies = []
tfs = []

Nc = float(b.shape[1])

for i in tqdm(range(b.shape[0])):
    
    unique, _counts = np.unique(b[i], return_counts=True)
    
    uniques.append(unique)
    counts.append(_counts)
    
    qcs = _counts[_counts>0]/Nc
    ln_qcs = np.log(qcs)
    
    entropy = -1 * np.sum(qcs * ln_qcs)   
    entropies.append(entropy)

100%|██████████| 2034183/2034183 [01:12<00:00, 28043.64it/s]


In [61]:
len(entropies)

2034183

In [62]:
len(list(set(entropies)))

89502

In [63]:
vectorized_words.shape[0]

249

In [75]:
# average term frequencies
term_frequencies_ = vectorized_words.sum(axis=0).reshape(-1,).tolist()[0]
term_frequencies = list(np.array(term_frequencies_)/vectorized_words.shape[0])

In [67]:
len([(index, freq) for index,freq in enumerate(term_frequencies) if freq>10])

22094

In [76]:
good_words_with_indices = [(index, freq) for index,freq in enumerate(term_frequencies) if freq>10]

In [81]:
good_word_tfs = [np.log(freq) for (index,freq) in good_words_with_indices]
good_indices = [index for index, freq in good_words_with_indices]
good_word_entropies = [entropies[index] for index in good_indices]

In [82]:
len(good_word_entropies)

22094

__Plotting entropy to term frequency__

In [80]:
trace0 = go.Scatter(
    x = good_word_entropies,
    y = good_word_tfs,
    mode = 'markers',
    text = words,
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)


layout= go.Layout(
    title= 'Entropy - Tf plot',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Entropy',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Term Frequency',
        ticklen= 5,
        gridwidth= 2,
    ),
    
    showlegend= False
)

plotdata = [trace0]


fig= go.Figure(data=plotdata, layout=layout)
py.iplot(fig)

/usr/local/lib/python3.6/dist-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

